## ***Modelado News-Clustering***

In [1]:
#!pip install umap-learn==0.5.6

In [2]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import plotly.express as px
from umap import UMAP
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.mixture import GaussianMixture
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score, davies_bouldin_score

url_base = "https://raw.githubusercontent.com/lacamposm/Metodos-Estadisticos/main/data/"

In [3]:
def silhouette_scorer(estimator, X):
    """
    Calcula el Silhouette Score utilizando la métrica de coseno para evaluar la calidad de los clústeres.

    El Silhouette Score mide cómo de similar es un punto de datos a otros puntos de su mismo clúster en comparación
    con los puntos de otros clústeres. El valor oscila entre -1 y 1, donde un valor más alto indica que los puntos
    están mejor agrupados y más separados de otros clústeres.

    :param estimator: Un modelo de clustering que debe tener un método `predict` para obtener las etiquetas de los
    clústeres de las muestras de `X`.
    :param X: Un array-like o matriz de datos (n_samples, n_features) sobre los cuales se calcularán los clústeres.
    :return: El Silhouette Score para el conjunto de datos `X` y las etiquetas de los clústeres obtenidos del
    `estimator`. Se utiliza la métrica de coseno para calcular las distancias.
    """
    if not hasattr(estimator, "predict"):
        raise ValueError(f"El estimator: {estimator} no tiene metodo 'predict'")

    labels = estimator.predict(X)
    return silhouette_score(X, labels, metric="cosine")


def davies_bouldin_scorer(estimator, X):
    """
    Calcula el índice de Davies-Bouldin para evaluar la calidad de los clústeres.

    El índice de Davies-Bouldin mide la media del cociente entre las distancias intra-clúster e inter-clúster para cada
    clúster. Un valor más bajo del índice indica una mejor formación de los clústeres, ya que minimiza la dispersión
    dentro de los clústeres y maximiza la separación entre ellos.

    :param estimator: Un modelo de clustering que debe tener un método `predict` para obtener las etiquetas de los
    clústeres de las muestras de `X`.
    :param X: Un array-like o matriz de datos (n_samples, n_features) sobre los cuales se calcularán los clústeres.
    :return: El valor negativo del índice de Davies-Bouldin para el conjunto de datos `X` y las etiquetas de los
    clústeres obtenidos del `estimator`. Esto invierte la métrica para que pueda maximizarse en las búsquedas
    de hiperparámetros.
    """
    if not hasattr(estimator, "predict"):
        raise ValueError(f"El estimator: {estimator} no tiene metodo 'predict'")

    labels = estimator.predict(X)
    score = -davies_bouldin_score(X, labels)
    return score

In [4]:
def transform_dict_best_model(input_dict):
    """
    Transforma un diccionario de parámetros de un modelo en un nuevo formato más estructurado.

    Esta función toma un diccionario que contiene información sobre un modelo, incluyendo un preprocesador, un método
    de reducción de dimensiones y un agrupador (clusterer), junto con sus parámetros específicos. La función reorganiza
    esta información en un formato más accesible y fácil de usar.

    :param input_dict: Un diccionario que contiene los parámetros del modelo. Debe incluir las claves "preprocessor",
    "dim_reduction", y "clusterer", así como pares clave-valor para los parámetros del agrupador y la reducción de
    dimensiones que siguen el formato "clusterer__<param_name>" y "dim_reduction__<param_name>".
    :return: Un nuevo diccionario estructurado que contiene el preprocesador, la reducción de dimensiones, el agrupador,
    y los parámetros asociados a cada uno. La clave de los parámetros de agrupador y    de reducción de dimensiones
    se simplifica eliminando el prefijo correspondiente.
    """
    return {
        "preprocessor": input_dict["preprocessor"],
        "dim_reduction": input_dict["dim_reduction"],
        "clusterer": input_dict["clusterer"],
        "clusterer_params": {
            k.split("__")[-1]: v for k, v in input_dict.items()
            if k.startswith("clusterer__")
        },
        "dim_reduction_params": {
            k.split("__")[-1]: v for k, v in input_dict.items()
            if k.startswith("dim_reduction__")
        }
    }
    

def tsne_plot_2d(df, estimator=None, scorer=None, metric=None):
    """
    Genera un gráfico 2D utilizando t-SNE para visualizar embeddings en un espacio reducido.

    Esta función aplica un preprocesador a un DataFrame, seguido de una reducción de dimensionalidad mediante t-SNE.
    Se pueden visualizar los embeddings resultantes en un gráfico, con la opción de agregar etiquetas de clúster para
    distinguir diferentes grupos en los datos.

    :param df: DataFrame que contiene los datos a visualizar. Cada fila representa una observación y cada columna
    representa una característica o variable.
    :param estimator:
    :param scorer:
    :param metric
    """
    if estimator is not None:
        params_estimator = transform_dict_best_model(estimator.get_params())
        preprocessor = params_estimator.get("preprocessor")
        dim_reduction = params_estimator.get("dim_reduction")
        clusterer = params_estimator.get("clusterer")
        best_score = scorer(estimator, df)

        if isinstance(dim_reduction, PCA):
            pipeline = Pipeline([
                ("preprocessor", preprocessor),
                ("dim_reduction", dim_reduction),
            ])

            normalized_embeddings = pipeline.fit_transform(df)

        else:
            normalized_embeddings = preprocessor.fit_transform(df)

        tsne = TSNE(n_components=2, random_state=42, metric=metric)
        df_plot = pd.DataFrame(tsne.fit_transform(normalized_embeddings), columns=["tSNE1", "tSNE2"])
        df_plot["cluster"] = estimator.predict(df) + 1
        df_plot = df_plot.sort_values(by=["cluster"])
        df_plot["cluster"] = df_plot["cluster"].astype("string")
        title = (
            f"""<b>Clustering: News-Summary-Embeddings in Low dimension with t-SNE</b><br>"""
            f"""<span style='font-size: 11px;'>Scaler: {preprocessor.__class__.__name__}, """
            f"""Dim-Reduction: {dim_reduction.__class__.__name__}, Estimator: {clusterer.__class__.__name__}, Metric """
            f"""Plot: {metric}, Scorer Metric: {scorer.__name__}={best_score:.3f}</span>"""
        )
        color = "cluster"

    else:
        df_scaled = StandardScaler().fit_transform(df)
        tsne = TSNE(n_components=2, random_state=42, metric=metric)
        df_plot = pd.DataFrame(tsne.fit_transform(df_scaled), columns=["tSNE1", "tSNE2"])
        title = (
            f"""<b>News-Summary-Embeddings in Low dimension with t-SNE</b><br>"""
            f"""<span style='font-size: 10px;'>Scaler: {StandardScaler().__class__.__name__}"""
        )
        color = None

    (
        px.scatter(
            df_plot, x="tSNE1", y="tSNE2", color=color,
            title=title,
            opacity=0.8,
            color_discrete_sequence=px.colors.qualitative.Dark24,
            template="plotly_white"
        )
        .update_traces(marker=dict(size=3))
        .show()
    )
    

def umap_plot_2d(df, estimator=None, scorer=None, metric=None):
    """
    Función que genera un gráfico en 2D utilizando UMAP y muestra los clústeres.

    :param df: pd.DataFrame con los embeddings de los resumenes de las noticias).
    :param estimator:
    :param scorer:
    :param metric
    """
    if estimator is not None:
        params_estimator = transform_dict_best_model(estimator.get_params())
        preprocessor = params_estimator.get("preprocessor")
        dim_reduction = params_estimator.get("dim_reduction")
        clusterer = params_estimator.get("clusterer")
        best_score = scorer(estimator, df)

        if isinstance(dim_reduction, UMAP):
            df_scaled = preprocessor.fit_transform(df)
            dim_reduction.n_components = 2
            df_plot = pd.DataFrame(dim_reduction.fit_transform(df_scaled), columns=["UMAP1", "UMAP2"])
        else:
            pipeline = Pipeline([
                ("preprocessor", preprocessor),
                ("dim_reduction", dim_reduction),
            ])
            df_scaled = pipeline.fit_transform(df)
            reducer = UMAP(n_components=2, random_state=42, metric=metric)
            df_plot = pd.DataFrame(reducer.fit_transform(df_scaled), columns=["UMAP1", "UMAP2"])

        df_plot["cluster"] = estimator.predict(df) + 1
        df_plot = df_plot.sort_values(by=["cluster"])
        df_plot["cluster"] = df_plot["cluster"].astype("string")
        color = "cluster"
        title = (
            f"""<b>Clustering: News-Summary-Embeddings in Low dimension with UMAP</b><br>"""
            f"""<span style='font-size: 11px;'>Scaler: {preprocessor.__class__.__name__}, """
            f"""Dim-Reduction: {dim_reduction.__class__.__name__}, Estimator: {clusterer.__class__.__name__}, Metric:"""
            f""" {metric}, Scorer Metric: {scorer.__name__}={best_score:.3f}</span>"""

        )

    else:
        df_scaled = StandardScaler().fit_transform(df)
        reducer = UMAP(n_components=2, random_state=42)
        df_plot = pd.DataFrame(reducer.fit_transform(df_scaled), columns=["UMAP1", "UMAP2"])
        title = (
            f"""<b>News-Summary-Embeddings in Low dimension with UMAP</b><br>"""
            f"""<span style='font-size: 10px;'>Scaler: {StandardScaler().__class__.__name__}"""
        )
        color = None

    (
        px.scatter(
            df_plot, x="UMAP1", y="UMAP2", color=color,
            title=title,
            opacity=0.8,
            color_discrete_sequence=px.colors.qualitative.Dark24,
            template="plotly_white"
        )
        .update_traces(marker=dict(size=3))
        .show()
    )

In [5]:
n_iter = 10                                     # Número de ajustes en RandomizedSearchCV.
scorers = ["silhouette", "davies-bouldin"]      # Puedes seleccionar diferentes metrica de adecuación de clusters
random_state = 42                               # Semilla aletoria de RandomizedSearchCV para replicar resultados.


scoring_dict = {
    "silhouette": silhouette_scorer,
    "davies-bouldin": davies_bouldin_scorer,
}


list_scalers = [StandardScaler(), MinMaxScaler(), RobustScaler()]

pca_n_components = [num for num in range(5, 80)]
umap_metrics = ["cosine", "correlation"]
umap_n_neighbors = [5, 10, 15]
umap_min_dist = [0.1, 0.3, 0.5]
umap_n_components = [2, 3, 4]

gauss_mix_n_components = [num for num in range(5, 13)]
gauss_mix_covariance_type = ["full", "diag"]
kmeans_n_clusters = [num for num in range(5, 21)]

param_grid = [
    {
        "preprocessor": list_scalers,
        "dim_reduction": [PCA()],
        "dim_reduction__n_components": pca_n_components,
        "clusterer": [KMeans()],
        "clusterer__n_clusters": kmeans_n_clusters,
    },
    {
        "preprocessor": list_scalers,
        "dim_reduction": [PCA()],
        "dim_reduction__n_components": pca_n_components,
        "clusterer": [GaussianMixture()],
        "clusterer__n_components": gauss_mix_n_components,
        "clusterer__covariance_type": gauss_mix_covariance_type
    },
    {
        "preprocessor": list_scalers,
        "dim_reduction": [UMAP()],
        "dim_reduction__n_neighbors": umap_n_neighbors,
        "dim_reduction__min_dist": umap_min_dist,
        "dim_reduction__n_components": umap_n_components,
        "dim_reduction__metric": umap_metrics,
        "clusterer": [KMeans()],
        "clusterer__n_clusters": kmeans_n_clusters,
    },
    {
        "preprocessor": list_scalers,
        "dim_reduction": [UMAP()],
        "dim_reduction__n_neighbors": umap_n_neighbors,
        "dim_reduction__min_dist": umap_min_dist,
        "dim_reduction__n_components": umap_n_components,
        "dim_reduction__metric": umap_metrics,
        "clusterer": [GaussianMixture()],
        "clusterer__n_components": gauss_mix_n_components,
        "clusterer__covariance_type": gauss_mix_covariance_type
    }
]


pipeline = Pipeline([
        ("preprocessor", "passthrough"),
        ("dim_reduction", "passthrough"),
        ("clusterer", "passthrough")
    ])

random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_grid,
    scoring=scoring_dict,
    cv=2,
    n_jobs=-1,
    verbose=3,
    n_iter=n_iter,
    refit="silhouette",
    random_state=random_state
)

In [6]:
df_embed = pd.read_parquet(url_base + "mxbai-embed-large_summary_news_el_tiempo.parquet")
df_embed

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
url_page,,,,,,,,,,,,,,,,,,,,,
https://www.eltiempo.com/mundo/latinoamerica/evo-morales-amenaza-con-bloquear-bolivia-si-la-fiscalia-lo-captura-3389757,-0.020917,-0.006987,-0.026591,0.017555,0.032127,-0.046068,-0.048087,0.004656,0.005598,0.049441,...,0.028381,0.019583,0.007692,0.002656,0.050690,0.041930,0.001792,-0.013605,-0.002610,-0.016743
https://www.eltiempo.com/bogota/vehiculo-se-incendia-en-medio-de-la-carretera-en-fusagasuga-cundinamarca-bomberos-atienden-la-emergencia-3389756,-0.031880,0.027921,-0.016712,0.032918,0.024373,-0.010680,-0.058477,-0.027742,0.044773,0.041911,...,0.048250,0.040552,-0.037708,0.000493,0.010013,0.003768,0.026349,0.010262,-0.003502,0.014081
https://www.eltiempo.com/mundo/eeuu-y-canada/esta-es-la-edad-en-la-que-un-adulto-mayor-debe-dejar-de-conducir-harvard-3389502,-0.021267,0.007155,0.034475,0.030611,-0.008942,-0.016378,-0.011049,-0.016557,0.039848,0.030022,...,0.073989,0.016037,-0.018351,-0.029923,0.037829,-0.002450,-0.024424,0.009189,0.001148,-0.009378
https://www.eltiempo.com/mundo/europa/morire-en-prision-escribio-en-sus-memorias-el-opositor-ruso-alexei-navalni-3389755,-0.001724,-0.011231,0.006536,0.007330,0.013462,-0.022181,-0.013968,0.002475,0.010086,0.029852,...,0.043627,0.040211,-0.020006,0.009449,0.054445,0.034426,-0.022466,-0.010860,-0.025343,-0.013328
https://www.eltiempo.com/colombia/otras-ciudades/alcaldia-de-villa-de-leyva-en-boyaca-entrega-balance-tras-el-vendaval-de-ese-sabado-hay-12-personas-heridas-3389752,-0.015240,0.014905,0.014017,0.032451,0.006244,-0.013349,-0.052630,0.047903,0.035785,0.030179,...,-0.000029,0.047785,-0.012244,-0.007176,0.010913,0.012925,0.016821,0.001564,-0.004066,0.014183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://www.eltiempo.com/mas-contenido/el-llamado-desde-el-sur-del-meta-3385783,-0.015249,0.032793,-0.022702,0.048635,-0.002823,-0.039297,-0.053139,0.034664,0.032758,0.025721,...,0.027468,0.045833,-0.016872,-0.021523,0.025576,0.014999,-0.008812,-0.021739,-0.010349,0.009338
https://www.eltiempo.com/mundo/medio-oriente/asi-localizo-israel-al-lider-de-hezbola-los-detalles-del-operativo-de-inteligencia-para-efectuar-el-bombardeo-aereo-3385781,-0.013496,-0.014557,-0.018691,0.047176,0.018366,-0.011003,-0.093697,0.026669,-0.002839,-0.000249,...,0.032180,-0.000164,-0.033964,-0.035739,0.008483,0.055563,-0.048210,-0.003561,0.001201,-0.018963
https://www.eltiempo.com/mas-contenido/del-crudo-al-turismo-3385901,-0.024780,0.041507,0.002410,0.001920,-0.007972,0.006983,-0.026574,0.025248,0.020038,0.037295,...,0.042184,0.043891,-0.024159,-0.011424,0.054588,-0.006380,0.002650,-0.021769,-0.050148,0.003720


In [7]:
tsne_plot_2d(df_embed, metric="cosine")
umap_plot_2d(df_embed, metric="cosine")

In [8]:
best_model = random_search.fit(df_embed)

Fitting 2 folds for each of 10 candidates, totalling 20 fits
[CV 2/2] END clusterer=GaussianMixture(), clusterer__covariance_type=diag, clusterer__n_components=8, dim_reduction=PCA(), dim_reduction__n_components=68, preprocessor=MinMaxScaler(); davies-bouldin: (test=-3.141) silhouette: (test=-0.031) total time=   4.4s
[CV 2/2] END clusterer=GaussianMixture(), clusterer__covariance_type=full, clusterer__n_components=12, dim_reduction=PCA(), dim_reduction__n_components=10, preprocessor=MinMaxScaler(); davies-bouldin: (test=-3.661) silhouette: (test=0.065) total time=   3.3s
[CV 1/2] END clusterer=GaussianMixture(), clusterer__covariance_type=diag, clusterer__n_components=8, dim_reduction=PCA(), dim_reduction__n_components=68, preprocessor=MinMaxScaler(); davies-bouldin: (test=-3.638) silhouette: (test=0.081) total time=   3.9s
[CV 1/2] END clusterer=KMeans(), clusterer__n_clusters=8, dim_reduction=PCA(), dim_reduction__n_components=66, preprocessor=RobustScaler(); davies-bouldin: (test=-

In [9]:
best_model.best_estimator_

Pipeline(steps=[('preprocessor', RobustScaler()),
                ('dim_reduction', PCA(n_components=66)),
                ('clusterer', KMeans())])

In [10]:
pd.DataFrame(best_model.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_preprocessor,param_dim_reduction__n_neighbors,param_dim_reduction__n_components,param_dim_reduction__min_dist,param_dim_reduction__metric,param_dim_reduction,...,split0_test_silhouette,split1_test_silhouette,mean_test_silhouette,std_test_silhouette,rank_test_silhouette,split0_test_davies-bouldin,split1_test_davies-bouldin,mean_test_davies-bouldin,std_test_davies-bouldin,rank_test_davies-bouldin
0,61.451363,0.641376,107.538870,0.217128,MinMaxScaler(),15.0,3,0.5,cosine,UMAP(),...,0.071795,0.060500,0.066147,0.005648,8,-4.267171,-4.668142,-4.467657,0.200486,10
1,3.725260,0.844143,1.847641,0.498057,RobustScaler(),NaN,66,NaN,NaN,PCA(),...,0.111457,0.100532,0.105995,0.005462,1,-3.343701,-3.259551,-3.301626,0.042075,1
2,5.758554,0.342652,1.820870,0.283193,RobustScaler(),NaN,76,NaN,NaN,PCA(),...,0.075848,0.057526,0.066687,0.009161,7,-3.541835,-3.453316,-3.497576,0.044260,5
3,2.628685,0.579398,1.688401,0.410230,MinMaxScaler(),NaN,10,NaN,NaN,PCA(),...,0.093768,0.064748,0.079258,0.014510,4,-3.536984,-3.660859,-3.598921,0.061937,6
4,61.116615,0.601326,111.256128,0.370988,StandardScaler(),10.0,3,0.5,cosine,UMAP(),...,0.099070,0.068144,0.083607,0.015463,3,-3.981713,-3.735402,-3.858557,0.123156,8
5,62.002027,0.326131,111.037768,0.518493,MinMaxScaler(),15.0,2,0.3,cosine,UMAP(),...,0.080206,0.070261,0.075233,0.004973,6,-4.226512,-4.151934,-4.189223,0.037289,9
6,3.070148,0.086945,1.979107,0.178444,MinMaxScaler(),NaN,41,NaN,NaN,PCA(),...,0.056272,-0.069238,-0.006483,0.062755,10,-3.477332,-3.190424,-3.333878,0.143454,2
7,2.286027,0.184273,1.842155,0.061444,MinMaxScaler(),NaN,68,NaN,NaN,PCA(),...,0.081023,-0.031090,0.024966,0.056056,9,-3.637915,-3.141201,-3.389558,0.248357,4
8,1.472808,0.029106,1.822065,0.295383,MinMaxScaler(),NaN,10,NaN,NaN,PCA(),...,0.093114,0.087733,0.090423,0.002691,2,-3.795622,-3.490213,-3.642917,0.152704,7
9,3.484603,0.191882,1.100228,0.169730,MinMaxScaler(),NaN,55,NaN,NaN,PCA(),...,0.086151,0.065150,0.075651,0.010500,5,-3.353578,-3.393720,-3.373649,0.020071,3


In [11]:
labels = best_model.predict(df_embed)
labels

array([3, 2, 1, ..., 5, 5, 4], dtype=int32)

In [12]:
tsne_plot_2d(df_embed, best_model.best_estimator_, silhouette_scorer, "cosine")
umap_plot_2d(df_embed, best_model.best_estimator_, silhouette_scorer, "cosine")

In [13]:
pd.DataFrame(best_model.predict(df_embed) + 1, index=df_embed.index, columns=["cluster"])

,cluster
url_page,
https://www.eltiempo.com/mundo/latinoamerica/evo-morales-amenaza-con-bloquear-bolivia-si-la-fiscalia-lo-captura-3389757,4
https://www.eltiempo.com/bogota/vehiculo-se-incendia-en-medio-de-la-carretera-en-fusagasuga-cundinamarca-bomberos-atienden-la-emergencia-3389756,3
https://www.eltiempo.com/mundo/eeuu-y-canada/esta-es-la-edad-en-la-que-un-adulto-mayor-debe-dejar-de-conducir-harvard-3389502,2
https://www.eltiempo.com/mundo/europa/morire-en-prision-escribio-en-sus-memorias-el-opositor-ruso-alexei-navalni-3389755,5
https://www.eltiempo.com/colombia/otras-ciudades/alcaldia-de-villa-de-leyva-en-boyaca-entrega-balance-tras-el-vendaval-de-ese-sabado-hay-12-personas-heridas-3389752,3
...,...
https://www.eltiempo.com/mas-contenido/el-llamado-desde-el-sur-del-meta-3385783,6
https://www.eltiempo.com/mundo/medio-oriente/asi-localizo-israel-al-lider-de-hezbola-los-detalles-del-operativo-de-inteligencia-para-efectuar-el-bombardeo-aereo-3385781,7
https://www.eltiempo.com/mas-contenido/del-crudo-al-turismo-3385901,6


### ***Caracterizacion de los clusters***

In [14]:
import requests

response = requests.get(url_base + "octubre_news_summary.json")
data_json = response.json()

df_predict = pd.DataFrame(best_model.predict(df_embed) +1, index=df_embed.index, columns=["cluster"])
df_summary = pd.DataFrame(data=data_json.values(), index=data_json.keys(), columns=["summary"])
df_review = df_predict.merge(df_summary, left_index=True, right_index=True)
df_review

,cluster,summary
https://www.eltiempo.com/mundo/latinoamerica/evo-morales-amenaza-con-bloquear-bolivia-si-la-fiscalia-lo-captura-3389757,4,"Evo Morales, expresidente de Bolivia, amenaza ..."
https://www.eltiempo.com/bogota/vehiculo-se-incendia-en-medio-de-la-carretera-en-fusagasuga-cundinamarca-bomberos-atienden-la-emergencia-3389756,3,El Cuerpo de Bomberos de Cundinamarca atendió ...
https://www.eltiempo.com/mundo/eeuu-y-canada/esta-es-la-edad-en-la-que-un-adulto-mayor-debe-dejar-de-conducir-harvard-3389502,2,Los accidentes automovilísticos están en aumen...
https://www.eltiempo.com/mundo/europa/morire-en-prision-escribio-en-sus-memorias-el-opositor-ruso-alexei-navalni-3389755,5,"Alexei Navalni, opositor ruso al presidente Vl..."
https://www.eltiempo.com/colombia/otras-ciudades/alcaldia-de-villa-de-leyva-en-boyaca-entrega-balance-tras-el-vendaval-de-ese-sabado-hay-12-personas-heridas-3389752,3,Un fuerte vendaval sorprendió a los habitantes...
...,...,...
https://www.eltiempo.com/mas-contenido/el-llamado-desde-el-sur-del-meta-3385783,6,El acuerdo de paz con las Farc ha permitido la...
https://www.eltiempo.com/mundo/medio-oriente/asi-localizo-israel-al-lider-de-hezbola-los-detalles-del-operativo-de-inteligencia-para-efectuar-el-bombardeo-aereo-3385781,7,"El viernes, un bombardeo aéreo israelí eliminó..."
https://www.eltiempo.com/mas-contenido/del-crudo-al-turismo-3385901,6,El tema central es la transición energética en...
https://www.eltiempo.com/mas-contenido/el-imponente-tesoro-verde-3385938,6,"El Meta enfrenta la amenaza de deforestación, ..."


In [15]:
def get_text_sample_by_cluster(label_cluster, size_sample):
    """
    """
    df = df_review.query("cluster == @label_cluster").sample(size_sample)
    text_to_llm = df["summary"].to_list()
    text_to_llm = text_to_llm = "\n".join([f"Noticia {i+1}\n{news}\n" for i, news in enumerate(text_to_llm)])
    print(text_to_llm)
    return text_to_llm

print(get_text_sample_by_cluster(2, 30))

Noticia 1
El informe de la Sociedad Española de Gastroenterología, Hepatología y Nutrición Pediátrica enfatiza la importancia de una alimentación balanceada para mantener el buen estado de salud. Se destaca la linaza como superalimento que contribuye a prevenir enfermedades autoinmunes, especialmente la artritis. Expertos recomiendan conocer el valor nutricional de los alimentos y evitar el sedentarismo y el estrés.

Noticia 2
El consumo del mango africano (Irvingia gabonensis) ha ganado atención por sus propiedades para reducir la grasa corporal y combatir la fatiga, según estudios. Las semillas, ricas en fibra soluble y antioxidantes, han sido objeto de investigación y se utilizan como suplemento natural. Expertos destacan su potencial para mejorar la salud general, aunque recomiendan consultar con profesionales de la salud antes del consumo.

Noticia 3
La Sociedad Colombiana de Cardiología y Cirugía Cardiovascular destaca los beneficios del ejercicio físico para las mujeres, incluye

In [16]:
# Activar Ollama con el modelo qwen2.5:7b o con el que sea de su eleccion
# en este caso es recomendado uno mucho mas "capaz"

In [17]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

def get_topic_in_cluster(news: str, model_name: str = "qwen2.5:7b") -> str:

    llm_ollama = ChatOllama(
        model=model_name,
        temperature=0.1,
        num_predict=4096,
        num_ctx=16000,
        keep_alive=0
    )

    prompt_characterics = ChatPromptTemplate(
        [
            SystemMessagePromptTemplate.from_template(
                """Eres un asistente especializado en análisis de noticias y clustering semántico. Tu tarea es ayudar a identificar temas, """
                """palabras clave y subtemas dentro de grupos de resúmenes de noticias. Analizas los patrones comunes en los eventos, """
                """lugares, personas y palabras clave para proporcionar una descripción clara y precisa del tema principal de cada grupo """
                """de noticias. Además, eres capaz de detectar subtemas o enfoques recurrentes que puedan enriquecer el análisis."""
            ),
            HumanMessagePromptTemplate.from_template(
                """Los siguientes son resúmenes de noticias que han sido agrupados en un clúster debido a su similitud semántica. """
                """Tu tarea es analizar estos resúmenes y proporcionar una caracterización general del tema principal del clúster. """
                """Identifica patrones comunes en los eventos, lugares, personas y palabras clave mencionadas, y determina un título o """
                """categoría representativa que capture la esencia del clúster."""
                """Además, destaca si hay algún subtema relevante o recurrente que ayude a entender mejor el enfoque de las noticias en """
                """este grupo. Asegúrate de que la descripción sea clara y precisa para que refleje el contenido general de las noticias."""
                """Resúmenes de noticias en el clúster: {news}"""
                """Salida esperada:
                    Tema principal: Una breve descripción del tema general del clúster.
                    Palabras clave: Lista de palabras clave relevantes.
                    Subtemas (si los hay): Cualquier subtema adicional que aparezca en varios resúmenes.
                    Título sugerido: Un título breve que represente el clúster."""
                )
        ]
    )

    llm_chain = prompt_characterics | llm_ollama | StrOutputParser()

    return llm_chain.invoke({"news": news})

In [19]:
label_cluster = 1
print(f"Cluster con label: {label_cluster}\nNoticias de la muestra:\n")
news_summary = get_text_sample_by_cluster(label_cluster, 15)
print("\nRespuesta LLM:\n")
print(get_topic_in_cluster(news_summary))

Cluster con label: 1
Noticias de la muestra:

Noticia 1
David Alonso, piloto colombiano de Moto3, ganó su décimo triunfo en Japón, asegurando matemáticamente el título mundial. Su victoria emocionante incluyó celebraciones con banderas y la moto que usaba al comenzar. Alonso expresó emoción y gratitud por su equipo y familia. Este triunfo marca el primer campeonato mundial para Colombia en Moto3, con importantes implicaciones para el país y el deportista.

Noticia 2
El partido entre Bolivia y Colombia por la eliminatoria al Mundial del 2026 contará con el arbitraje central de Wilton Sampaio, un árbitro brasileño con experiencia en torneos internacionales. La designación fue realizada por la Conmebol para el encuentro en El Alto a 4.150 metros de altura. Sampaio ha dirigido partidos entre ambos países anteriormente y será apoyado por jueces brasileños, lo que podría influir en el resultado del juego.

Noticia 3
El Camp Nou, estadio del FC Barcelona, está sufriendo remodelaciones a gran 